In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image

In [11]:
from google.colab import drive
drive.mount('/content/drive')
import os

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
link = '/content/drive/MyDrive/Ativities/NextGenAI Camp/Onsite/Hackathon/dataset-nextgen-Day1'
train = pd.read_csv(os.path.join(link, 'train.csv'))
train = train.drop(columns='id')

In [22]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12295 entries, 0 to 12294
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Image   12295 non-null  object 
 1   MOS     12295 non-null  float64
 2   Class   12295 non-null  object 
dtypes: float64(1), object(2)
memory usage: 288.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12295 entries, 0 to 12294
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Image   12295 non-null  object 
 1   MOS     12295 non-null  float64
 2   Class   12295 non-null  object 
dtypes: float64(1), object(2)
memory usage: 288.3+ KB


In [23]:
temp = {}
for index, label in enumerate(train['Class'].unique()):
  temp[label] = index
train.Class = train['Class'].map(temp)

In [24]:
import json
json.dump(temp, open('label to index.json', 'w'))

In [25]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], # ImageNet
                [0.229, 0.224, 0.225])
])

class Image_Dataset(Dataset):
  def __init__(self, dataframe, transform=None):
    self.df = dataframe
    self.transform = transform

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    img_path = self.df.iloc[index]['Image']
    img = Image.open(link + '/' + img_path).convert('RGB')
    if self.transform:
      image = self.transform(img)

    mos = torch.tensor(self.df.iloc[index]['MOS'], dtype=torch.float32)
    label = torch.tensor(self.df.iloc[index]['Class'], dtype=torch.long)
    return image, mos, label

dataset = Image_Dataset(train, transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

In [26]:
class CNN(nn.Module):
  def __init__(self, num_label):
    super(CNN, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 32, 3, stride=1, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(32, 64, 3, stride=1, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2),

        nn.Conv2d(64, 128, 3, stride=1, padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.AdaptiveAvgPool2d((7, 7))
    )

    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(128 * 7 * 7, 256),
        nn.ReLU(),
        nn.Dropout(0.3)
    )

    self.regression = nn.Linear(256, 1) # MOS
    self.classification = nn.Linear(256, num_label) # label

  def forward(self, x):
    x = self.features(x)
    x = self.fc(x)
    mos = self.regression(x).squeeze(1)
    label = self.classification(x)
    return mos, label

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
num_label = len(train['Class'].unique())

model = CNN(num_label).to(device)
MSE = nn.MSELoss()
CrossEntropy = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epoch = 12
for i in range(0, epoch):
  model.train()
  total_loss = 0
  for img, mos, label in tqdm(dataloader, desc=f'Epoch {i+1} / {epoch} | Loss : {loss}', leave=False):
    img, mos, label = img.to(device), mos.to(device), label.to(device)

    mos_hat, label_hat = model(img)

    regression_loss = MSE(mos_hat, mos)
    classification_loss = CrossEntropy(label_hat, label)
    loss = regression_loss + (0.5*classification_loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

In [28]:
torch.save({
    'model_state': model.state_dict(),
    'class_to_index': temp
    }, 'CNN_model.pth')

In [29]:
final = os.path.join(link, 'test')
result = []
model = CNN(num_label).to(device)

x = torch.load('CNN_model.pth', map_location='cpu')
model.load_state_dict(x['model_state'])
label_to_index = x['class_to_index']
index_to_label = {}

for label_name, label_index in label_to_index.items():
  index_to_label[label_index] = label_name

model.eval()
for filename in tqdm(os.listdir(final), desc='Predicting'):
  if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
    img_path = os.path.join(final, filename)

    img = Image.open(img_path).convert('RGB')
    img = transform(img).unsqueeze(0)

    with torch.no_grad():
      mos_hat, label_hat = model(img)

    mos_hat = mos_hat.item()
    label_index = torch.argmax(label_hat, dim=1).item()
    label_name = index_to_label[label_index]

    id_img = filename.split('_')[-1].split('.')[0]

    result.append({
        'id' : int(id_img),
        'Predicted_MOS' : mos_hat,
        'Class' : label_name
    })

df = pd.DataFrame(result)
df = df.sort_values('id')
df.to_csv('My_Answer.csv', index=False)
df

Predicting: 100%|██████████| 2011/2011 [03:17<00:00, 10.19it/s]


,id,Predicted_MOS,Class
1209,11,0.469309,BooPadPongali
1464,13,0.497199,BooPadPongali
1786,31,0.404555,BooPadPongali
105,32,0.401142,GaengJued
11,37,0.459456,GaengJued
...,...,...,...
1675,14282,0.546173,BooPadPongali
1736,14283,0.504702,BooPadPongali
1584,14284,0.463414,GaengJued
1537,14285,0.444489,GaengJued


Predicting: 100%|██████████| 2011/2011 [02:43<00:00, 12.29it/s]


,id,Predicted_MOS,Class
1209,11,0.529296,CurriedFishCake
1464,13,0.526996,CurriedFishCake
1786,31,0.559845,KuayJab
105,32,0.565845,GaengJued
11,37,0.560593,KuayJab
...,...,...,...
1675,14282,0.534809,CurriedFishCake
1736,14283,0.543620,KaiJeowMooSaap
1584,14284,0.563740,GaengJued
1537,14285,0.559492,KuayJab
